Scratch code from the [low level intro](https://www.tensorflow.org/programmers_guide/low_level_intro)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

In [2]:
a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # impicitly tf.float32
total = a + b
print(a)
print(b)
print(total)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)
Tensor("add:0", shape=(), dtype=float32)


In [3]:
print("{} = {} + {}".format(total, a, b))

Tensor("add:0", shape=(), dtype=float32) = Tensor("Const:0", shape=(), dtype=float32) + Tensor("Const_1:0", shape=(), dtype=float32)


N.B. `tf.tensors` are just handles for values that will flow through the computation graph.

Each element in the graph is given a unique name. E.g., `Const:0`, `Const_1:0`, `add:0`. 

Use TensorBoard to visualize a computation graph.

In [4]:
writer = tf.summary.FileWriter('.')
writer.add_graph(tf.get_default_graph())

tensors are evaluated when the computation graph is run inside a `tf.Session`

In [5]:
sess = tf.Session()
print(sess.run(total))

7.0


In [6]:
print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


In [7]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
print(sess.run(vec))
print(sess.run(vec))
res = sess.run((out1, out2))
print("\n{}\n{}".format(res[0],res[1]))

[0.4092859 0.9459708 0.3737898]
[0.8291367  0.33590138 0.52889895]

[1.082676  1.004563  1.9091648]
[2.082676  2.0045629 2.909165 ]


Placeholders allow graphs to accept external inputs

In [8]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

In [9]:
print(sess.run(z, feed_dict={x:3, y:4.5}))
print(sess.run(z, feed_dict={x:[1,3], y:[2,4]}))

7.5
[3. 7.]


N.B. A placeholder can overwrite any tensor in the graph. The only real difference between placeholders and other tensors is that placeholders throw an error if no value is fed to them.

`Datasets` are the primary way of streaming data to a model. They encapsulate an iterable of tensors. 

In [10]:
my_data = [
    [0, 1,],
    [2, 3,],
    [4, 5,],
    [6, 7,]
]
print("{}\ntype: {}".format(my_data, type(my_data)))

slices = tf.data.Dataset.from_tensor_slices(my_data)
next_item = slices.make_one_shot_iterator().get_next()

print("\n{}\ntype: {}".format(next_item, type(next_item)))

[[0, 1], [2, 3], [4, 5], [6, 7]]
type: <class 'list'>

Tensor("IteratorGetNext:0", shape=(2,), dtype=int32)
type: <class 'tensorflow.python.framework.ops.Tensor'>


In [11]:
while True:
  try:
    print(sess.run(next_item))
  except tf.errors.OutOfRangeError:
    break

[0 1]
[2 3]
[4 5]
[6 7]


During a `tf.Session` any `tf.Tensor` may have only one value.
**Layers** are the preferred way of adding trainable parameters to a model. Layers encapsulate variables and the operations that run on them. *"For example a [densely-connected layer](https://developers.google.com/machine-learning/glossary/#fully_connected_layer) performs a weighted sum across all inputs for each output and applies an optional [activation function](https://developers.google.com/machine-learning/glossary/#activation_function)."*

In [12]:
x = tf.placeholder(tf.float32, shape=[None,3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)

# "init" is a handle for another tensorflow operation.
# It will initialize global variables when the session 
# is run but only those that exist in the graph before 
# the initializer is created.
init = tf.global_variables_initializer()
sess.run(init)

# after initialization,
print(sess.run(y, {x: [[1,2,3],[4,5,6]]}))

[[-1.1721561]
 [-2.8075485]]


Running the following (without an initializer) throws an uninitialized value error.

```
x = tf.placeholder(tf.float32, shape=[None,3])
linear_model = tf.layers.Dense(units=1)
y = linear_model(x)
print(sess.run(y, {x: [[1,2,3],[4,5,6]]}))
```

Layer "shortcut" functions allow running a layer without naming them, but this makes introspection and debugging more difficult.

In [13]:
x = tf.placeholder(tf.float32, shape=[None, 3])
y = tf.layers.dense(x, units=1)
init = tf.global_variables_initializer()
sess.run(init)
print(sess.run(y, {x: [[1,2,3],[4,5,6]]}))

[[1.8502705]
 [3.3839169]]


Feature columns

In [14]:
features = {
    'sales' : [[5], [10], [8], [9]],
    'department' : ['sports', 'sports', 'gardening', 'gardening']
}

department_column = tf.feature_column.categorical_column_with_vocabulary_list(
'department', ['sports', 'gardening'])
department_column = tf.feature_column.indicator_column(department_column)

columns = [
    tf.feature_column.numeric_column('sales'),
    department_column
]

inputs = tf.feature_column.input_layer(features, columns)

Columns are containers with state. They load values (e.g., from the features dictionary) and may require initialization. Categorical columns use lookup tables, which can be initialized with the `tf.tables_initializer`, as below.

In [15]:
var_init = tf.global_variables_initializer()
table_init = tf.tables_initializer()
sess = tf.Session()
sess.run((var_init, table_init))
print(sess.run(inputs))

[[ 1.  0.  5.]
 [ 1.  0. 10.]
 [ 0.  1.  8.]
 [ 0.  1.  9.]]


Train a small regression model manually

In [16]:
# define data and expected output values
x = tf.constant([[1], [2], [3], [4]], dtype=tf.float32)
actual_values = [[0], [-1], [-2], [-3]]
y_true = tf.constant(actual_values, dtype=tf.float32)

# define the model
linear_model = tf.layers.Dense(units=1)
y_pred = linear_model(x)

# evaluate predictions, before training
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

print("initial predictions:\n{}".format(sess.run(y_pred)))

# use mean squared error for the loss function
loss = tf.losses.mean_squared_error(labels=y_true, predictions=y_pred)

print("\nintial loss: {}".format(sess.run(loss)))

# construct a training function
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# the training function updates variables in the graph
# note that "train" is an "op" not a tensor, so it returns no value 
loss_values = []
for i in range(100):
    _, loss_value = sess.run((train, loss))
    loss_values.append(loss_value)
print("\nloss_values = {}".format(loss_values))

print("\nfinal predictions:\n{}".format(sess.run(y_pred)))

print("\nactual values:\n{}".format(actual_values))

initial predictions:
[[0.52570164]
 [1.0514033 ]
 [1.5771049 ]
 [2.1028066 ]]

intial loss: 10.829732894897461

loss_values = [10.829733, 7.6074333, 5.3709927, 3.8186212, 2.7409148, 1.9925715, 1.4727699, 1.111551, 0.86037326, 0.6855536, 0.5637204, 0.47865665, 0.41910943, 0.37727094, 0.3477232, 0.3267068, 0.31161314, 0.30063233, 0.29250824, 0.2863695, 0.2816113, 0.27781394, 0.2746864, 0.27202645, 0.2696939, 0.26759148, 0.26565164, 0.26382744, 0.26208645, 0.26040584, 0.2587701, 0.25716832, 0.25559276, 0.25403827, 0.2525012, 0.25097883, 0.24946943, 0.24797185, 0.24648502, 0.24500835, 0.24354145, 0.24208392, 0.24063562, 0.2391962, 0.23776561, 0.23634368, 0.2349304, 0.23352563, 0.23212934, 0.23074137, 0.22936177, 0.22799043, 0.22662729, 0.22527231, 0.22392541, 0.2225866, 0.22125578, 0.21993288, 0.21861796, 0.2173109, 0.2160116, 0.2147201, 0.21343634, 0.21216017, 0.21089175, 0.20963086, 0.20837751, 0.20713167, 0.20589323, 0.20466226, 0.20343864, 0.20222226, 0.20101318, 0.19981135, 0.19861674